In [33]:
import fitbit
import gather_keys_oauth2 as Oauth2
import pandas as pd
import datetime
from datetime import timedelta
import numpy as np
import warnings
import json
import requests
warnings.filterwarnings('ignore')

# Load the JSON file
with open('config.json', 'r') as config_file:
    config = json.load(config_file)

    

# Correctly access client_id and client_secret
CLIENT_ID = config['client_id']
CLIENT_SECRET = config['client_secret']
ACCESS_TOKEN = config['access_token']

# Assuming CLIENT_ID, CLIENT_SECRET, and ACCESS_TOKEN are already set
header = {'Authorization': 'Bearer {}'.format(ACCESS_TOKEN)}


In [34]:
DATE_FORMAT = '%Y-%m-%d'
    
def prettyPrintDate(date):
    return date.strftime(DATE_FORMAT);


def safeGet(obj, key, defaultVal = np.nan):
    return obj.get(key, defaultVal)  


def filterArrObj(arrList, keyName, keyValue):
    for arrItem in arrList:
        if arrItem.get(keyName) == keyValue:
            return arrItem
    return {}

# Define the endpoints
profile_url = "https://api.fitbit.com/1/user/-/profile.json"
activities_url = "https://api.fitbit.com/1/user/-/activities/date/2023-12-17.json"
sleep_url = "https://api.fitbit.com/1.2/user/-/sleep/date/2023-12-17.json"
heart_rate_url = "https://api.fitbit.com/1/user/-/activities/heart/date/2023-12-17/1d.json"

# Function to make a GET request
def get_fitbit_data(url):
    response = requests.get(url, headers=header)
    return response.json()


def getActivities(date):
    activitiyResponse = get_fitbit_data(f"https://api.fitbit.com/1/user/-/activities/date/{date}.json")
    
    activitySummary = activitiyResponse.get('summary',{})
    activityData = {
        'Calories Burned':safeGet(activitySummary,'caloriesOut'),
        'Calories BMR': safeGet(activitySummary,'caloriesBMR'),
        'Resting Heart Rate':safeGet(activitySummary,'restingHeartRate'),
        'Steps':safeGet(activitySummary,'steps'),
        'Distance (Km)':filterArrObj(activitySummary.get('distances', []), 'activity', 'total').get('distance', np.nan),
        'Elevation (Ft)':activitySummary.get('elevation',{}),
        'Floors':activitySummary.get('floors',{}),
        'Minutes Sedentary':activitySummary.get('sedentaryMinutes',{}),
        'Minutes Lightly Active':activitySummary.get('lightlyActiveMinutes',{}),
        'Minutes Fairly Active':activitySummary.get('fairlyActiveMinutes',{}),
        'Minutes Very Active':activitySummary.get('veryActiveMinutes',{}),
        'Activity Calories': activitySummary.get('activityCalories',{}),
        'Active Score': activitySummary.get('activeScore',{}),
        'Cardio minutes': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Cardio').get('minutes', np.nan),
        'Cardio calories': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Cardio').get('caloriesOut', np.nan),
        'Fat Burn minutes': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Fat Burn').get('minutes', np.nan),
        'Fat Burn calories': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Fat Burn').get('caloriesOut', np.nan),
        'Peak minutes': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Peak').get('minutes', np.nan),
        'Peak calories': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Peak').get('caloriesOut', np.nan),
        'Normal Cardio minutes': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Out of Range').get('minutes', np.nan),
        'Normal Cardio calories': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Out of Range').get('caloriesOut', np.nan),
    }
    
    
    return activityData
def getSleep(date):
    sleepResponse = get_fitbit_data(f"https://api.fitbit.com/1/user/-/sleep/date/{date}.json")
    
    sleepData = {}
    for sleepLog in sleepResponse.get('sleep', []):
        if sleepLog.get('isMainSleep'):
            sleepLevelsSummary = sleepResponse.get('summary', {}).get('stages', {})

            sleepData['Sleep Efficiency'] = safeGet(sleepLog, 'efficiency')
            sleepData['Minutes Asleep'] = safeGet(sleepLog, 'minutesAsleep')
            sleepData['Minutes to fall asleep'] = safeGet(sleepLog, 'minutesToFallAsleep')
            sleepData['Sleep Start time'] = safeGet(sleepLog, 'startTime')
            sleepData['Sleep End time'] = safeGet(sleepLog, 'endTime')
            
            sleepData['Time in bed'] = safeGet(sleepLog, 'timeInBed')

            sleepData['Minutes Deep sleep'] = sleepLevelsSummary.get('deep', 0)
            sleepData['Deep sleep count'] = sleepLevelsSummary.get('deep', {})
    #             sleepData['% Deep sleep'] = percent(safeGet(sleepData, 'Minutes Deep sleep', 0)/safeGet(sleepData, 'Time in bed', 0))
            
            sleepData['Minutes Light sleep'] = sleepLevelsSummary.get('light', 0)
            sleepData['Light sleep count'] = sleepLevelsSummary.get('light', {})
    #             sleepData['% Light sleep'] = percent(safeGet(sleepData, 'Minutes Light sleep', 0)/safeGet(sleepData, 'Time in bed', 0))
            
            sleepData['Minutes REM sleep'] = sleepLevelsSummary.get('rem', 0)
            sleepData['REM sleep count'] = sleepLevelsSummary.get('rem', {})
    #             sleepData['% REM sleep'] = percent(safeGet(sleepData, 'Minutes REM sleep', 0)/safeGet(sleepData, 'Time in bed', 0))
            
            sleepData['Minutes Asleep'] = sleepData['Minutes Deep sleep'] + sleepData['Minutes Light sleep'] + sleepData['Minutes REM sleep']
            sleepData['Minutes Awake'] = sleepLevelsSummary.get('wake', {})
            sleepData['Minutes Awake count'] = sleepLevelsSummary.get('wake', {})      
    return sleepData

def getDateData(date):
    weekDayNum = date.isoweekday()
    return {
        'Day of Week': weekDayNum,
        'Is Weekday': weekDayNum<6,
        'Is Weekend': weekDayNum>5,
        'Date': prettyPrintDate(date)
        };

def fetchAllData(date):
    dateStr = prettyPrintDate(date)
    
    print("Fetching fitbit data for: " + dateStr)
    
    nextDate = date + timedelta(days=1)
    sleepData = getSleep(prettyPrintDate(nextDate))
     
    activitiesData = getActivities(dateStr)
    dateData = getDateData(date)
    
    
    mergedData = {**sleepData, **activitiesData, **dateData} #merging the dictionaries together
    return mergedData

In [35]:
# initiating an empty dataframe to store the final results
database = pd.DataFrame()


def fetchAndAppendToDb(date, database):    
    mergedData = fetchAllData(date)
    for key, value in mergedData.items():
        if value == {}:
            mergedData[key] = np.nan
    # Convert mergedData to DataFrame and then concatenate
    print(mergedData)
    mergedData_df = pd.DataFrame([mergedData])
    database = pd.concat([database, mergedData_df], ignore_index=True)
    return database

def fetchData(database):    
    #here we are extracting the data for the last 1 week
    StartDate = (datetime.datetime.today() - datetime.timedelta(90)) #date from which you want to start extracting data
    EndDate = (datetime.datetime.today() - datetime.timedelta(0)) #date upto which you want to extract data 
    dateToFetch = StartDate + timedelta(days=1)
    while dateToFetch < EndDate:
        database = fetchAndAppendToDb(dateToFetch, database)
        dateToFetch = dateToFetch + timedelta(days=1)
    print("----------------------------------------------")
    print("Data fill completed! 👍👍")
    return database



In [36]:
database = fetchData(database)
database

Fetching fitbit data for: 2023-09-20


{'Calories Burned': 1795, 'Calories BMR': 1795, 'Resting Heart Rate': nan, 'Steps': 0, 'Distance (Km)': 0, 'Elevation (Ft)': 0, 'Floors': 0, 'Minutes Sedentary': 1440, 'Minutes Lightly Active': 0, 'Minutes Fairly Active': 0, 'Minutes Very Active': 0, 'Activity Calories': 0, 'Active Score': -1, 'Cardio minutes': nan, 'Cardio calories': nan, 'Fat Burn minutes': nan, 'Fat Burn calories': nan, 'Peak minutes': nan, 'Peak calories': nan, 'Normal Cardio minutes': nan, 'Normal Cardio calories': nan, 'Day of Week': 3, 'Is Weekday': True, 'Is Weekend': False, 'Date': '2023-09-20'}
Fetching fitbit data for: 2023-09-21
{'Calories Burned': 1795, 'Calories BMR': 1795, 'Resting Heart Rate': nan, 'Steps': 0, 'Distance (Km)': 0, 'Elevation (Ft)': 0, 'Floors': 0, 'Minutes Sedentary': 1440, 'Minutes Lightly Active': 0, 'Minutes Fairly Active': 0, 'Minutes Very Active': 0, 'Activity Calories': 0, 'Active Score': -1, 'Cardio minutes': nan, 'Cardio calories': nan, 'Fat Burn minutes': nan, 'Fat Burn calories

,Calories Burned,Calories BMR,Resting Heart Rate,Steps,Distance (Km),Elevation (Ft),Floors,Minutes Sedentary,Minutes Lightly Active,Minutes Fairly Active,...,Sleep End time,Time in bed,Minutes Deep sleep,Deep sleep count,Minutes Light sleep,Light sleep count,Minutes REM sleep,REM sleep count,Minutes Awake,Minutes Awake count
0,1795,1795,NaN,0,0.00,0.0,0,1440,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1795,1795,NaN,0,0.00,0.0,0,1440,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1795,1795,NaN,0,0.00,0.0,0,1440,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2609,1795,NaN,4001,3.00,0.0,0,1018,397,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2399,1795,NaN,1496,1.12,0.0,0,1082,358,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,1795,1795,NaN,0,0.00,0.0,0,1440,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,1795,1795,NaN,0,0.00,0.0,0,1440,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87,1795,1795,NaN,0,0.00,0.0,0,1440,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,1795,1795,NaN,0,0.00,0.0,0,1440,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
database.to_excel('Fitbit_data.xlsx')